# Getting Started with PrimeHub Python SDK
PrimeHub Python SDK makes you automation with PrimeHub Platform.

In order to make the SDK working, you have to

- install the library with pip
- create a config file in the ~/.primehub/config.json

# Part 1: prerequisite: Configure the environment.

## 1. Install with pip
Let's install PrimeHub Python SDK with pip.

In [1]:
!pip install primehub-python-sdk

## 2. Request the API Token
In order to get the token, you have to have an account in the PrimeHub cluster, the following process will ask you loing with your account.



In [2]:
# PLEASE UPDATE PRIMEHUB_CLUSTER to your cluster
PRIMEHUB_CLUSTER = 'http://primehub-python-sdk.primehub.io'

In [3]:
import os
from primehub import PrimeHub, PrimeHubConfig

ph = PrimeHub(PrimeHubConfig())
if not os.path.isfile(os.path.join(os.getenv("HOME"), ".primehub/config.json")):
    ph.config.generate(PRIMEHUB_CLUSTER)

In [4]:
ph = PrimeHub(PrimeHubConfig())
if ph.is_ready():
    print("PrimeHub Python SDK setup successfully")
    print("Current Group:", ph.primehub_config.current_group)
else:
    print("PrimeHub Python SDK couldn't get the group information, please check the configuration.")

PrimeHub Python SDK setup successfully
Current Group: {'id': 'ccdd52b8-db16-46ce-8a06-70343c79e82a', 'name': 'phusers', 'displayName': 'primehub users'}


## 3. Check the account is Admin account

Use `ph.me.me` to know that the account is admin account.

In [5]:
account_information = ph.me.me()
account_information['isAdmin']

True

## 4. Clean up the previous testing data.

We need to clean up the previous data to prevent the notebook error,

In [6]:
import pandas as pd

In [7]:
# Clean up 'gitsync-secret-create-secret-by-sdk' secret data.

df_list = pd.DataFrame.from_records(list(ph.admin.secrets.list()))
if len(df_list) > 0:
    target_id = list(df_list.loc[df_list['name'] == 'create-secret-by-sdk']["id"])

    if len(target_id) == 1:
        print("clean the secret data: {}".format(target_id))
        ph.admin.secrets.delete("gitsync-secret-create-secret-by-sdk")

clean the secret data: ['gitsync-secret-create-secret-by-sdk']


In [8]:
# Clean up 'my-custom-image' image data.

df_list = pd.DataFrame.from_records(list(ph.admin.images.list()))
if len(df_list) > 0:
    target_id = list(df_list.loc[df_list['name'] == 'my-custom-image']["id"])

    if len(target_id) == 1:
        print("clean the image data: {}".format(target_id))
        ph.admin.images.delete('my-custom-image')

clean the image data: ['my-custom-image']


In [9]:
# Clean up 'image-by-sdk` image data.

df_list = pd.DataFrame.from_records(list(ph.admin.images.list()))
if len(df_list) > 0:
    target_id = list(df_list.loc[df_list['name'] == 'image-by-sdk']["id"])

    if len(target_id) == 1:
        print("clean the image data: {}".format(target_id))
        ph.admin.images.delete('image-by-sdk')

clean the image data: ['image-by-sdk']


In [10]:
# Clean up "test_group_from_jupyter" group data.

df_list = pd.DataFrame.from_records(list(ph.admin.groups.list()))
if len(df_list) > 0:
    target_id = list(df_list.loc[df_list['name'] == "test_group_from_jupyter"]["id"])

    if len(target_id) == 1:
        print("clean the group data: {}".format(target_id))
        ph.admin.groups.delete(list(target_id)[0])

clean the group data: ['ba3d19ad-38a7-498b-99eb-44e0509a18b9']


# Part 2: Image-wise operation
We will test:
    
- Create an image record with a existing image and a pull secret
- Disable the Global and assign the image to multiple groups
- Create another image record by building a custom image

## 1. Create new secrets
Use ` ph.admin.secrets.create` to create the pull secret.

In [11]:
secret = ph.admin.secrets.create(dict(name='create-secret-by-sdk', type='opaque', secret='keep it secret'))
secret

{'id': 'gitsync-secret-create-secret-by-sdk'}

## 2. Create new image
Use `ph.admin.images.create` to create the jupyter image.

In [12]:
# Create an image
config = {
  "name": "image-by-sdk",
  "displayName": "Learning how to create an image from SDK",
  "description": "base-notebook with python 3.7",
  "type": "both",
  "url": "infuseai/docker-stacks:base-notebook-63fdf50a",
  "urlForGpu": "infuseai/docker-stacks:base-notebook-63fdf50a-gpu",
  "global": True,
  "useImagePullSecret": secret['id']
}
image = ph.admin.images.create(config)
image

{'id': 'image-by-sdk'}

## 3. Create another image record by building a custom image
Use `ph.admin.images.create` to create the jupyter image and add imageSpec to custom create image.

In [13]:
# Create an custom image
config = {
  "name": "my-custom-image",
  "type": "both",
  "global": True,
  "imageSpec": {
    "baseImage": "jupyter/base-notebook",
    "packages": {
      "pip": [
        "primehub-python-sdk"
      ]
    }
  }
}
image_custom = ph.admin.images.create(config)

In [14]:
output = ph.admin.images.get(image_custom['id'])
del output['logEndpoint']

output

{'id': 'my-custom-image',
 'name': 'my-custom-image',
 'displayName': 'my-custom-image',
 'description': None,
 'type': 'both',
 'url': None,
 'urlForGpu': None,
 'useImagePullSecret': '',
 'global': True,
 'groups': [],
 'isReady': False,
 'imageSpec': {'baseImage': 'jupyter/base-notebook',
  'pullSecret': None,
  'packages': {'apt': [], 'conda': [], 'pip': ['primehub-python-sdk']}},
 'jobStatus': None}

## 4. Disable the Global and assign the instance type to multiple groups

First, we use `ph.admin.groups.create` to create the group and Use `ph.admin.instancetypes.update` to:
- Disable global instance
- Assign the instance type to multiple groups

In [15]:
# Create a group with admin role
config = {
  "name": "test_group_from_jupyter",
  "displayName": "test_group",
  "enabledDeployment": False,
  "enabledSharedVolume": False,
  "quotaCpu": 0.5,
  "quotaGpu": 0,
  "admins": "",
  "users": {
      "connect": []
  }
}
data = ph.admin.groups.create(config)

In [16]:
import pandas as pd
df_list = pd.DataFrame.from_records(list(ph.admin.groups.list()))
df_list

,id,displayName,name,admins,quotaCpu,quotaGpu,quotaMemory,projectQuotaCpu,projectQuotaGpu,projectQuotaMemory,sharedVolumeCapacity
0,ccdd52b8-db16-46ce-8a06-70343c79e82a,primehub users,phusers,phadmin,NaN,NaN,None,None,None,None,1.0
1,fa382829-d08a-4421-8de4-82e3231fe7a7,test_group,test_group_from_jupyter,,0.5,0.0,None,None,None,None,NaN


In [17]:
instance_connect = []
for list_id in list(df_list['id'])[0:2]:
    instance_connect.append({"id": list_id})
instance_connect

[{'id': 'ccdd52b8-db16-46ce-8a06-70343c79e82a'},
 {'id': 'fa382829-d08a-4421-8de4-82e3231fe7a7'}]

In [18]:
# Update the image information.
config = {
  "global": False,
  "groups": {
    "connect": instance_connect
  },
}
output = ph.admin.images.update(image['id'], config)
del output['logEndpoint']

output

{'id': 'image-by-sdk',
 'name': 'image-by-sdk',
 'displayName': 'Learning how to create an image from SDK',
 'description': 'base-notebook with python 3.7',
 'type': 'both',
 'url': 'infuseai/docker-stacks:base-notebook-63fdf50a',
 'urlForGpu': 'infuseai/docker-stacks:base-notebook-63fdf50a-gpu',
 'useImagePullSecret': None,
 'global': False,
 'groups': [{'id': 'ccdd52b8-db16-46ce-8a06-70343c79e82a',
   'name': 'phusers',
   'displayName': 'primehub users'},
  {'id': 'fa382829-d08a-4421-8de4-82e3231fe7a7',
   'name': 'test_group_from_jupyter',
   'displayName': 'test_group'}],
 'isReady': True,
 'imageSpec': None,
 'jobStatus': None}

In [19]:
output = ph.admin.images.get(image['id'])
del output['logEndpoint']

output

{'id': 'image-by-sdk',
 'name': 'image-by-sdk',
 'displayName': 'Learning how to create an image from SDK',
 'description': 'base-notebook with python 3.7',
 'type': 'both',
 'url': 'infuseai/docker-stacks:base-notebook-63fdf50a',
 'urlForGpu': 'infuseai/docker-stacks:base-notebook-63fdf50a-gpu',
 'useImagePullSecret': None,
 'global': False,
 'groups': [{'id': 'ccdd52b8-db16-46ce-8a06-70343c79e82a',
   'name': 'phusers',
   'displayName': 'primehub users'},
  {'id': 'fa382829-d08a-4421-8de4-82e3231fe7a7',
   'name': 'test_group_from_jupyter',
   'displayName': 'test_group'}],
 'isReady': True,
 'imageSpec': None,
 'jobStatus': None}